In [ ]:
class RequestHandler:
    """
    Handles HTTP requests with proper error handling, retries, and resource management
    """
    
    def __init__(self, 
                 base_url: str, 
                 auth_token: Optional[str] = None, 
                 timeout: int = 30,
                 max_retries: int = 3,
                 retry_delay: float = 1.0,
                 max_backoff: float = 60.0):
        """
        Initialize request handler
        
        Args:
            base_url: Base URL for API requests
            auth_token: Authentication token (optional)
            timeout: Request timeout in seconds
            max_retries: Maximum number of retries
            retry_delay: Initial delay for retries (seconds)
            max_backoff: Maximum backoff time (seconds)
        """
        self.base_url = base_url
        self.auth_token = auth_token
        self.timeout = timeout
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.max_backoff = max_backoff
        self.session: Optional[aiohttp.ClientSession] = None
    
    async def __aenter__(self):
        """Initialize session when entering context"""
        self.session = aiohttp.ClientSession()
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """Close session when exiting context"""
        await self.close()
    
    def _get_session(self) -> aiohttp.ClientSession:
        """Get the current session or create a new one"""
        if self.session is None or self.session.closed:
            self.session = aiohttp.ClientSession()
        return self.session
    
    def _get_headers(self, additional_headers: Optional[Dict[str, str]] = None) -> Dict[str, str]:
        """Get request headers with authentication"""
        headers = {
            "Accept": "application/json, text/plain, */*",
            "Content-Type": "application/json; charset=utf-8",
            "User-Agent": "FluentAI/1.0"
        }
        
        if self.auth_token:
            headers["Cookie"] = f"GSSSO={self.auth_token}"
        
        if additional_headers:
            headers.update(additional_headers)
            
        return headers
    
    @log_request
    async def get(self, 
                 endpoint: str, 
                 params: Optional[Dict[str, Any]] = None, 
                 headers: Optional[Dict[str, str]] = None) -> str:
        """
        Make a GET request with retries
        
        Args:
            endpoint: API endpoint
            params: Query parameters
            headers: Additional headers
            
        Returns:
            Response text
        """
        url = f"{self.base_url}{endpoint}"
        req_headers = self._get_headers(headers)
        
        for attempt in range(self.max_retries + 1):
            try:
                session = self._get_session()
                async with session.get(
                    url, 
                    params=params, 
                    headers=req_headers, 
                    timeout=self.timeout
                ) as response:
                    await self._check_response(response)
                    return await response.text()
            
            except RateLimitError:
                if attempt >= self.max_retries:
                    logger.error(f"Rate limit exceeded, max retries reached: {url}")
                    raise
                    
                backoff = min(
                    self.max_backoff, 
                    self.retry_delay * (2 ** attempt)
                )
                logger.warning(f"Rate limit exceeded, retrying in {backoff:.2f}s: {url}")
                await asyncio.sleep(backoff)
                
            except (aiohttp.ClientError, asyncio.TimeoutError) as e:
                if attempt >= self.max_retries:
                    logger.error(f"Request failed after {self.max_retries} retries: {url}")
                    raise ContentFetchError(f"Failed to fetch content: {str(e)}")
                    
                backoff = min(
                    self.max_backoff, 
                    self.retry_delay * (2 ** attempt)
                )
                logger.warning(f"Request error, retrying in {backoff:.2f}s: {url}")
                await asyncio.sleep(backoff)
    
    @log_request
    async def post(self, 
                  endpoint: str, 
                  data: Any, 
                  headers: Optional[Dict[str, str]] = None) -> str:
        """
        Make a POST request with retries
        
        Args:
            endpoint: API endpoint
            data: Request data (will be JSON encoded)
            headers: Additional headers
            
        Returns:
            Response text
        """
        url = f"{self.base_url}{endpoint}"
        req_headers = self._get_headers(headers)
        
        # Convert data to JSON if not already a string
        if not isinstance(data, str):
            data = json.dumps(data)
        
        for attempt in range(self.max_retries + 1):
            try:
                session = self._get_session()
                async with session.post(
                    url, 
                    data=data, 
                    headers=req_headers, 
                    timeout=self.timeout
                ) as response:
                    await self._check_response(response)
                    return await response.text()
            
            except RateLimitError:
                if attempt >= self.max_retries:
                    logger.error(f"Rate limit exceeded, max retries reached: {url}")
                    raise
                    
                backoff = min(
                    self.max_backoff, 
                    self.retry_delay * (2 ** attempt)
                )
                logger.warning(f"Rate limit exceeded, retrying in {backoff:.2f}s: {url}")
                await asyncio.sleep(backoff)
                
            except (aiohttp.ClientError, asyncio.TimeoutError) as e:
                if attempt >= self.max_retries:
                    logger.error(f"Request failed after {self.max_retries} retries: {url}")
                    raise ContentFetchError(f"Failed to fetch content: {str(e)}")
                    
                backoff = min(
                    self.max_backoff, 
                    self.retry_delay * (2 ** attempt)
                )
                logger.warning(f"Request error, retrying in {backoff:.2f}s: {url}")
                await asyncio.sleep(backoff)
    
    async def _check_response(self, response: aiohttp.ClientResponse) -> None:
        """
        Check response for errors
        
        Args:
            response: API response
            
        Raises:
            RateLimitError: If rate limit is exceeded
            APIError: If response status is not 200
        """
        if response.status == 429:
            retry_after = response.headers.get("Retry-After", "60")
            raise RateLimitError(f"Rate limit exceeded. Retry after {retry_after}s")
            
        if response.status == 401:
            raise APIError(response.status, "Authentication failed")
            
        if response.status == 403:
            raise APIError(response.status, "Permission denied")
            
        if response.status == 404:
            raise APIError(response.status, "Resource not found")
            
        if response.status >= 400:
            error_text = await response.text()
            raise APIError(response.status, f"Request failed: {error_text[:200]}")
    
    async def close(self) -> None:
        """Close the session and release resources"""
        if self.session and not self.session.closed:
            await self.session.close()
            self.session = None
